# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_keyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

We create a table, put CSV data into the table and query it using CQL.

Because there can only be one song for a combination of item in session index and session id, we set the primary key to `(session_id, item_in_session)`.

In [8]:
# Creating a table for Query #1 using CQL.

query1 = "CREATE TABLE IF NOT EXISTS by_session_id_and_item_in_session"
query1 = query1 + "(session_id int, item_in_session int, artist text, song_title text, song_length float,\
                    PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query1)
except Exception as e:
    print(e)        

In [9]:
# Putting CSV data into Cassandra table.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO by_session_id_and_item_in_session(session_id,\
                            item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [16]:
# Querying the data.

select_query_1 = "SELECT session_id, item_in_session, artist, song_title, song_length FROM by_session_id_and_item_in_session\
                  WHERE session_id = 338\
                  AND item_in_session = 4"
try:
    rows = session.execute(select_query_1)
except Exception as e:
    print(e)

    
query_1_prettytable = PrettyTable()

query_1_prettytable.field_names = ["Session ID", "Item in session", "Artist", "Song", "Song length"]

for row in rows:
    query_1_prettytable.add_row([row.session_id, row.item_in_session, row.artist, row.song_title, row.song_length])

print(query_1_prettytable)

+------------+-----------------+-----------+---------------------------------+--------------------+
| Session ID | Item in session |   Artist  |               Song              |    Song length     |
+------------+-----------------+-----------+---------------------------------+--------------------+
|    338     |        4        | Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+------------+-----------------+-----------+---------------------------------+--------------------+


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We create a table, put CSV data into the table and query it using CQL.

Here, we set the primary key to `(user_id, session_id, item_in sesison)`, in which `item_in_sesison` is a clustering column.

For a set `user_id` and `session_id`, there are one or more songs being listened to during that session. We sort these songs by `item_in_session` by using this variable as a clustering column.

In [19]:
# Creating table for query 2.

query_2 = "CREATE TABLE IF NOT EXISTS by_user_id_and_session_id\
            (user_id int, session_id int, item_in_session int, artist text, song_title text,\
            first_name text, last_name text,\
            PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    session.execute(query_2)
except Exception as e:
    print(e)   

In [20]:
# Add data to by_user_id_and_session_id table

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO by_user_id_and_session_id(user_id, session_id,\
                            item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]),\
                                str(line[9]), str(line[1]), str(line[4])))

In [21]:
# Use select statement to verify the data was inserted into by_user_id_and_session_id table.

select_query_2 = "SELECT artist, song_title, first_name, last_name FROM by_user_id_and_session_id\
                  WHERE user_id = 10\
                  AND session_id = 182"

try:
    rows = session.execute(select_query_2)
except Exception as e:
    print(e)
    
query2_prettytable = PrettyTable()

query2_prettytable.field_names = ["Artist", "Song", "First name", "Last name"]
    
for row in rows:
    query2_prettytable.add_row([row.artist, row.song_title, row.first_name, row.last_name])

print(query2_prettytable)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First name | Last name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We want to identify any user who ever listened to a specific song. This can be uniquely identified by `(song_title, user_id)`. 

If a user listens to the same song again, this observation will not be inserted again into the table because Cassandra does not support duplicate rows. This is not a problem because we are not interested in the amount of times a user listened to the song, only whether he or she listened to it or not.

In [24]:
query_3 = "CREATE TABLE IF NOT EXISTS user_by_song(song_title text, user_id int,\
                                                   first_name text, last_name text,\
                                                   PRIMARY KEY(song_title, user_id))"

# I corrected the primary key according to reviewer feedback.

try:
    session.execute(query_3)
except Exception as e:
    print(e)

In [25]:
# Add data to user_by_song table

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song(song_title, user_id,\
                            first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]) )

In [27]:
# Use select statement to verify the data was inserted into user_by_song table.

select_query_3 = "SELECT first_name, last_name FROM user_by_song\
                  WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(select_query_3)
except Exception as e:
    print(e)

query3_prettytable = PrettyTable()

query3_prettytable.field_names = ["First name", "Last name"]
    
for row in rows:
    query3_prettytable.add_row([row.first_name, row.last_name])

print(query3_prettytable)

+------------+-----------+
| First name | Last name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [28]:
try:
    session.execute("DROP TABLE by_session_id_and_item_in_session")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE by_user_id_and_session_id")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE user_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()